#### IMPORTING DATA

In [57]:
import pandas as pd
import numpy as np

# Load the data from the CSV file
data = pd.read_csv('employee.csv')

# Print the data
data.head()


,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot
0,Bachelors,2017,Bangalore,3,34,Male,No,0,0
1,Bachelors,2013,Pune,1,28,Female,No,3,1
2,Bachelors,2014,New Delhi,3,38,Female,No,2,0
3,Masters,2016,Bangalore,3,27,Male,No,5,1
4,Masters,2017,Pune,3,24,Male,Yes,2,1


In [58]:
data.info()

# OUR DATATYPE FOR TOTAL_CHARGES IS NOT CORRECT

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4653 entries, 0 to 4652
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Education                  4653 non-null   object
 1   JoiningYear                4653 non-null   int64 
 2   City                       4653 non-null   object
 3   PaymentTier                4653 non-null   int64 
 4   Age                        4653 non-null   int64 
 5   Gender                     4653 non-null   object
 6   EverBenched                4653 non-null   object
 7   ExperienceInCurrentDomain  4653 non-null   int64 
 8   LeaveOrNot                 4653 non-null   int64 
dtypes: int64(5), object(4)
memory usage: 327.3+ KB


In [59]:
# Splitting features and target variable
X = data.drop(columns=['LeaveOrNot'])
y = data['LeaveOrNot']

#### CLASSIC EXAMPLE OF CUSTOM CODE

#### SELECTING CATEGORICAL AND NUMERICAL FOR PROCESSING

In [60]:
cat_feat = X.select_dtypes(include=[object]).columns

cat_feat= list(cat_feat.difference(['LeaveOrNot']))

print('\n','Categorical Features','\n', cat_feat,'\n')


 Categorical Features 
 ['City', 'Education', 'EverBenched', 'Gender'] 



In [61]:
numerical_features = list(X.select_dtypes(include=[np.float64,np.int64]))

print('\n','Numerical Features','\n', numerical_features,'\n')


 Numerical Features 
 ['JoiningYear', 'PaymentTier', 'Age', 'ExperienceInCurrentDomain'] 



#### SPLITTING DATA FOR TRAIN / TEST

In [62]:
from sklearn.model_selection import train_test_split

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [63]:
print('Train Data','\n',y_train.value_counts(normalize=True),'\n','\n','Test Data','\n', y_test.value_counts(normalize=True))

Train Data 
 LeaveOrNot
0    0.656368
1    0.343632
Name: proportion, dtype: float64 
 
 Test Data 
 LeaveOrNot
0    0.655209
1    0.344791
Name: proportion, dtype: float64


#### EXPORTING FEATURE INPUT METADATA

In [64]:
def summarize_cat(data,cat_feat):
  results=[]

  for column in data[cat_feat]:
      # Get the unique members of the column
      members = data[column].unique().tolist()
      # Append the column name and its unique members to the results list
      results.append([column, members])

  return pd.DataFrame(results, columns=['Column Name', 'Members'])

# Create a DataFrame from the results list
summarize_cat(X_train,cat_feat)

,Column Name,Members
0,City,"[New Delhi, Bangalore, Pune]"
1,Education,"[Masters, Bachelors, PHD]"
2,EverBenched,"[No, Yes]"
3,Gender,"[Male, Female]"


In [65]:
summarize_cat(data,cat_feat).to_dict()

{'Column Name': {0: 'City', 1: 'Education', 2: 'EverBenched', 3: 'Gender'},
 'Members': {0: ['Bangalore', 'Pune', 'New Delhi'],
  1: ['Bachelors', 'Masters', 'PHD'],
  2: ['No', 'Yes'],
  3: ['Male', 'Female']}}

In [66]:
# EXPORTING FOR DE

feat_dict = {'CATEGORICAL' : summarize_cat(data,cat_feat).to_dict(), 'NUMERICAL' : {'Column Name': numerical_features}}

feat_dict

{'CATEGORICAL': {'Column Name': {0: 'City',
   1: 'Education',
   2: 'EverBenched',
   3: 'Gender'},
  'Members': {0: ['Bangalore', 'Pune', 'New Delhi'],
   1: ['Bachelors', 'Masters', 'PHD'],
   2: ['No', 'Yes'],
   3: ['Male', 'Female']}},
 'NUMERICAL': {'Column Name': ['JoiningYear',
   'PaymentTier',
   'Age',
   'ExperienceInCurrentDomain']}}

In [67]:
feat_dict

{'CATEGORICAL': {'Column Name': {0: 'City',
   1: 'Education',
   2: 'EverBenched',
   3: 'Gender'},
  'Members': {0: ['Bangalore', 'Pune', 'New Delhi'],
   1: ['Bachelors', 'Masters', 'PHD'],
   2: ['No', 'Yes'],
   3: ['Male', 'Female']}},
 'NUMERICAL': {'Column Name': ['JoiningYear',
   'PaymentTier',
   'Age',
   'ExperienceInCurrentDomain']}}

In [68]:
import pickle

# save dictionary to person_data.pkl file
with open('feature_dict.pkl', 'wb') as fp:
    pickle.dump(feat_dict, fp)
    print('dictionary saved successfully to file')

dictionary saved successfully to file


#### CREATING THE PIPELINE

In [69]:
from sklearn.pipeline import Pipeline

# PREPROCESSING TRANSFORMATIONS ARE DONE ON EXAMPLE BASIS
# REAL WORLD SELECTION OF PREPROCSSING TRANSFORMATIONS MUST BE LOGICAL

from sklearn.preprocessing import FunctionTransformer

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

pipeline_num = Pipeline(steps=[
    ('scale_data', StandardScaler()),
    ('simple_imputer1', SimpleImputer(strategy='constant',fill_value=0)),
])

from sklearn.preprocessing import OneHotEncoder

pipeline_cat = Pipeline(steps=[
    ('OneHotEncode', OneHotEncoder(handle_unknown="ignore"))
])

from sklearn.compose import ColumnTransformer

preprocessor_stage_1 = ColumnTransformer(
    transformers=[
        ('cat', pipeline_cat, cat_feat),  # Categorical columns
        ('num', pipeline_num, numerical_features),     # Numerical columns
    ],remainder='drop')

preprocessor_stack = Pipeline(steps=[
    ('preprocessor_stage_1', preprocessor_stage_1)
])

# BECAUSE WE DIDN'T SPECIFY CUSTOMERID IN ANY OF CATEGORICAL OR NUMERICAL FEATURES (REMAINDER='drop') REMOVE IT OUT OF PIPELINE

In [70]:
preprocessor_stack

Pipeline(steps=[('preprocessor_stage_1',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('OneHotEncode',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['City', 'Education',
                                                   'EverBenched', 'Gender']),
                                                 ('num',
                                                  Pipeline(steps=[('scale_data',
                                                                   StandardScaler()),
                                                                  ('simple_imputer1',
                                                                   SimpleImputer(fill_value=0,
                                                                                 strategy='constant'))]),
                                                  ['JoiningYear', 'PaymentTier',
                                                   'Age',
                                                   'ExperienceInCurrentDomain'])]))])

#### FITTING THE PIPELINE

In [71]:
preprocessor_stack.fit(X_train)

Pipeline(steps=[('preprocessor_stage_1',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('OneHotEncode',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['City', 'Education',
                                                   'EverBenched', 'Gender']),
                                                 ('num',
                                                  Pipeline(steps=[('scale_data',
                                                                   StandardScaler()),
                                                                  ('simple_imputer1',
                                                                   SimpleImputer(fill_value=0,
                                                                                 strategy='constant'))]),
                                                  ['JoiningYear', 'PaymentTier',
                                                   'Age',
                                                   'ExperienceInCurrentDomain'])]))])

In [72]:
pd.DataFrame(preprocessor_stack.transform(X_train),columns=preprocessor_stack[-1].get_feature_names_out())

,cat__City_Bangalore,cat__City_New Delhi,cat__City_Pune,cat__Education_Bachelors,cat__Education_Masters,cat__Education_PHD,cat__EverBenched_No,cat__EverBenched_Yes,cat__Gender_Female,cat__Gender_Male,num__JoiningYear,num__PaymentTier,num__Age,num__ExperienceInCurrentDomain
0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,-1.109740,0.539424,0.117526,-1.861550
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,-1.645675,0.539424,-0.914641,0.061305
2,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.034001,-1.226396,-0.088907,-0.579646
3,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,-1.645675,0.539424,-1.121074,-0.579646
4,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.034001,-1.226396,1.149693,-0.579646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3717,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,-0.573805,0.539424,0.530393,-1.220598
3718,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,-1.109740,0.539424,-0.708208,0.702257
3719,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.498065,0.539424,1.975427,-1.220598
3720,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,-0.037870,0.539424,2.388294,-1.220598


In [73]:
y_train

2850    1
589     0
2086    0
445     0
3654    0
       ..
4426    0
466     0
3092    0
3772    0
860     1
Name: LeaveOrNot, Length: 3722, dtype: int64

In [74]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor_stack),
    ('classifier', RandomForestClassifier())
])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('preprocessor_stage_1',
                                  ColumnTransformer(transformers=[('cat',
                                                                   Pipeline(steps=[('OneHotEncode',
                                                                                    OneHotEncoder(handle_unknown='ignore'))]),
                                                                   ['City',
                                                                    'Education',
                                                                    'EverBenched',
                                                                    'Gender']),
                                                                  ('num',
                                                                   Pipeline(steps=[('scale_data',
                                                                                    StandardScaler()),
                                                                                   ('simple_imputer1',
                                                                                    SimpleImputer(fill_value=0,
                                                                                                  strategy='constant'))]),
                                                                   ['JoiningYear',
                                                                    'PaymentTier',
                                                                    'Age',
                                                                    'ExperienceInCurrentDomain'])]))])),
                ('classifier', RandomForestClassifier())])

In [75]:
X_train.head()

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain
2850,Masters,2013,New Delhi,3,30,Male,No,0
589,Bachelors,2012,Bangalore,3,25,Male,No,3
2086,Masters,2017,Pune,2,29,Male,No,2
445,Masters,2012,Pune,3,24,Male,No,2
3654,Masters,2017,New Delhi,2,35,Male,No,2


In [76]:
# Checking Training Accuracy
y_train_pred = pipeline.predict(X_train)

# Evaluate the model
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Accuracy:", accuracy_score(y_train,y_train_pred))
print("\nClassification Report:\n", classification_report(y_train,y_train_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_train,y_train_pred))

Accuracy: 0.9279957012358947

Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.97      0.95      2443
           1       0.94      0.84      0.89      1279

    accuracy                           0.93      3722
   macro avg       0.93      0.91      0.92      3722
weighted avg       0.93      0.93      0.93      3722


Confusion Matrix:
 [[2376   67]
 [ 201 1078]]


#### BREAKING PIPEPLINE INTO EXPLAINABLE PARTS ON TEST

In [81]:
# CREATING A TEST

pred_array=X_test.iloc[15:16:]

pred_array

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain
1128,Bachelors,2015,Pune,2,26,Female,No,4


In [82]:
print(preprocessor_stack)


Pipeline(steps=[('preprocessor_stage_1',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('OneHotEncode',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['City', 'Education',
                                                   'EverBenched', 'Gender']),
                                                 ('num',
                                                  Pipeline(steps=[('scale_data',
                                                                   StandardScaler()),
                                                                  ('simple_imputer1',
                                                                   SimpleImputer(fill_value=0,
                                                                                 strategy='constant'))]),
                                                

In [83]:
pd.DataFrame(
    preprocessor_stack.transform(pred_array),
    columns=preprocessor_stack.named_steps['preprocessor_stage_1'].get_feature_names_out()
)


,cat__City_Bangalore,cat__City_New Delhi,cat__City_Pune,cat__Education_Bachelors,cat__Education_Masters,cat__Education_PHD,cat__EverBenched_No,cat__EverBenched_Yes,cat__Gender_Female,cat__Gender_Male,num__JoiningYear,num__PaymentTier,num__Age,num__ExperienceInCurrentDomain
0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,-0.03787,-1.226396,-0.708208,0.702257


In [84]:
pred_array.to_json()

'{"Education":{"1128":"Bachelors"},"JoiningYear":{"1128":2015},"City":{"1128":"Pune"},"PaymentTier":{"1128":2},"Age":{"1128":26},"Gender":{"1128":"Female"},"EverBenched":{"1128":"No"},"ExperienceInCurrentDomain":{"1128":4}}'

In [85]:
# USING PIPELINE TO DO ALL TOGHETHER (PREPROCESSING FOLLOWED BY MODEL PREDICT)

# SINGLE PREDICTION

y_pred = pipeline.predict(pred_array)

y_pred


array([1])

In [86]:
# USING PIPELINE TO DO ALL TOGHETHER (PREPROCESSING FOLLOWED BY MODEL PREDICT)

# MULTIPLE PREDICTION
y_test_pred = pipeline.predict(X_test)

# EVALUATE MODEL FOR TEST ACCURACY SINCE WE HAVE TEST SET
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Accuracy:", accuracy_score(y_test, y_test_pred))
print("\nClassification Report:\n", classification_report(y_test, y_test_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_test_pred))

Accuracy: 0.8517722878625135

Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.92      0.89       610
           1       0.83      0.72      0.77       321

    accuracy                           0.85       931
   macro avg       0.84      0.82      0.83       931
weighted avg       0.85      0.85      0.85       931


Confusion Matrix:
 [[562  48]
 [ 90 231]]


### EXPORTING THE PIPELINE

In [87]:
!pip install dill

In [88]:
import dill

# save trained pipeline file

with open('pipeline.pkl', 'wb') as file:
    dill.dump(pipeline, file)

print('pipeline saved successfully to file')

pipeline saved successfully to file


### IMPORTING THE PIPELINE & TESTING PREDICTION

In [89]:
import dill

#Load the saved pipeline from the file

with open('pipeline.pkl', 'rb') as file:
    loaded_pipeline = dill.load(file)

print('pipeline loaded successfully to file')

pipeline loaded successfully to file


In [90]:
loaded_pipeline.__getstate__()

{'steps': [('preprocessor',
   Pipeline(steps=[('preprocessor_stage_1',
                    ColumnTransformer(transformers=[('cat',
                                                     Pipeline(steps=[('OneHotEncode',
                                                                      OneHotEncoder(handle_unknown='ignore'))]),
                                                     ['City', 'Education',
                                                      'EverBenched', 'Gender']),
                                                    ('num',
                                                     Pipeline(steps=[('scale_data',
                                                                      StandardScaler()),
                                                                     ('simple_imputer1',
                                                                      SimpleImputer(fill_value=0,
                                                                                    strategy=

In [91]:
y_pred = loaded_pipeline.predict(pred_array)

y_pred

array([1])